In [1]:
### ### 滑條############################
"""
hit_pos:
[0]:hit time
[1]:position(x,y,x,y,x,y......)
[2]:lebel
[3]:slider time
[4]:slider lenth
[5] slider type
"""
import os, pygame as pg
from pygame.locals import *
import sys
from random import *
import time
import numpy as np
import cv2 as plt
#########################fuctions#################################
def Ceq(posC,r,pos_kick):
    return(pos_kick[0]-posC[0])**2+(pos_kick[1]-posC[1])**2<r**2
def inx(start,end,var,arr):
    for i in range(start,end):
        if i==var:
            return 1
    return 0
def B(t,p):
    if t==1:return p[-1]
    c=1
    v=0
    n=len(p)-1
    t_=t/(1-t)
    tt=(1-t)**n
    for i in range(n+1):
        v+=c*tt*p[i]
        tt*=t_
        c*=(n-i)/(i+1)
    return v
def P(t,px,py):
    delta=4*((px[0]-px[1])*(py[1]-py[2])-(px[1]-px[2])*(py[0]-py[1]))
    if delta==0:
        return t*(px[2]-px[0])+px[0],t*(py[2]-py[0])+py[0]
    else:
        global Cx,Cy,p_rad,theta,mod_the
        if t==0:
            deltax=2*((px[0]**2+py[0]**2-px[1]**2-py[1]**2)*(py[1]-py[2])-(px[1]**2+py[1]**2-px[2]**2-py[2]**2)*(py[0]-py[1]))
            deltay=-2*((px[0]**2+py[0]**2-px[1]**2-py[1]**2)*(px[1]-px[2])-(px[1]**2+py[1]**2-px[2]**2-py[2]**2)*(px[0]-px[1]))
            Cx,Cy=deltax/delta,deltay/delta
            p_rad=((px[0]-Cx)**2+(py[0]-Cy)**2)**0.5
            roundx=[Cx+p_rad*np.cos(the) for the in np.linspace(0,2*np.pi,200)]
            roundy=[Cy+p_rad*np.sin(the) for the in np.linspace(0,2*np.pi,200)]

            theta=[]
            for i in range(3):
                if px[i]-Cx<0:
                    theta.append(np.arctan((py[i]-Cy)/(px[i]-Cx))+np.pi)
                else:
                    theta.append(np.arctan((py[i]-Cy)/(px[i]-Cx)))
                if theta[i]<0:
                    theta[i]+=2*np.pi
            arx=[]
            mod_the=theta[0]
            theta=np.array(theta)-mod_the
            states=list(theta[1:3]>0)
            if states==[0,0]:          #b<0 c<0
                if theta[1]<theta[2]:  #b<c
                    theta[1:3]+=2*np.pi            #b>c
            elif states==[0,1]:
                theta[2]-=2*np.pi      #b<0 c>0
            elif states==[1,0]:
                theta[2]+=2*np.pi      #b>0 c<0
            elif states==[1,1]:
                if theta[1]>theta[2]:  #b>0 c>0 
                    theta[1:3]-=2*np.pi
        return Cx+p_rad*np.cos(t*(theta[2]-theta[0])+theta[0]+mod_the),Cy+p_rad*np.sin(t*(theta[2]-theta[0])+theta[0]+mod_the)
def place_hit_object(dtt,s_type,hit_object):

    if s_type==0:
        adj=1+(dtt+par.appear/par.apr)/par.emerge
        if len(hit_object[1])>2:
            sli=int(hit_object[4]*par.precisity)
            if hit_object[5]=='B'or hit_object[5]=='L':
                for t in np.linspace(0,adj,sli):
                    bg.blit(body[int(adj*29)],(B(t,hit_object[1][::2])-wid/2,B(t,hit_object[1][1::2])-wid/2))
                for t in np.linspace(0,adj,sli):
                    bg.blit(heart[int(adj*29)],(B(t,hit_object[1][::2])-wid/4,B(t,hit_object[1][1::2])-wid/4))
            elif hit_object[5]=='P':
                for t in np.linspace(0,adj,sli):
                    bg.blit(body[int(adj*29)],(np.array(P(t,hit_object[1][::2],hit_object[1][1::2]))-wid/2))
                for t in np.linspace(0,adj,sli):
                    bg.blit(heart[int(adj*29)],(np.array(P(t,hit_object[1][::2],hit_object[1][1::2]))-wid/4))
            bg.blit(hita[int(adj*29)],(hit_object[1][0]-wid/2,hit_object[1][1]-wid/2))
        else:
            bg.blit(hita[int(adj*29)],(hit_object[1][0]-wid/2,hit_object[1][1]-wid/2))
    elif s_type==1:
        if len(hit_object[1])>2:
            sli=int(hit_object[4]*par.precisity)
            if hit_object[5]=='B'or hit_object[5]=='L':
                for t in np.linspace(0,1,sli):
                    bg.blit(slider_body,(B(t,hit_object[1][::2])-wid/2,B(t,hit_object[1][1::2])-wid/2))
                for t in np.linspace(0,1,sli):
                    bg.blit(slider_heart,(B(t,hit_object[1][::2])-wid/4,B(t,hit_object[1][1::2])-wid/4))
            elif hit_object[5]=='P':
                for t in np.linspace(0,1,sli):
                    bg.blit(slider_body,(np.array(P(t,hit_object[1][::2],hit_object[1][1::2]))-wid/2))
                for t in np.linspace(0,1,sli):
                    bg.blit(slider_heart,(np.array(P(t,hit_object[1][::2],hit_object[1][1::2]))-wid/4))
            bg.blit(hit,(hit_object[1][0]-wid/2,hit_object[1][1]-wid/2))
        else:
            bg.blit(hit,(hit_object[1][0]-wid/2,hit_object[1][1]-wid/2))
    bg.blit(default[hit_object[2]],(hit_object[1][0]-28,hit_object[1][1]-24))
########################classes##########################
class state:
    def __init__(self):
        self.select_song=0
        self.select_level=1
        self.gaming=2
        self.fail=3
        self.passed=4
        self.setup=5
class parameters:
    def __init__(self):
        self.dpi=3
        self.apr=6
        self.apcsize=128
        
        self.precisity=0.07
        
        self.map_modify=6
        self.range=1.3
        self.posx=300
        self.posy=200
        
        #time
        self.emerge=500
        self.label_fade=600
        self.fade=400
        self.perfect=100
        self.good=200
        self.appear=4000
        self.continues=200
state=state()
par=parameters()
##################### loading map ############################
hit_pos=[]
reg=[]
k=1
for i in os.listdir("sou"):
    if i=="skin":
        continue
    reg.append(i)
    print("%d. "%k,i)
    k+=1
song=reg[int(input("chose song"))-1]
reg=[]
k=1
for i in os.listdir("sou\\"+song):
    try:
        if i.split('.')[-1]=="osu":
            reg.append(i)
            print("%d."%k,i)
            k+=1
    except:
        pass
level=reg[int(input("chose level: "))-1]
reg=[]
file=open("sou\\"+song+"\\"+level,"r",encoding="utf-8")

def map_load():
    k=0
    cou=0
    last_pos=[-1,-1]
    last_time=0
    leb_n=1
    for i in file.read().split('[HitObjects]\n')[1].split('\n')[:-2]:
        x=i.split(',')
        
        if last_pos==[int(x[0]),int(x[1])]:
            k+=par.map_modify
        else:
            k=0
        if int(x[2])-last_time>par.continues:
            leb_n=1
        else:
            leb_n+=1
        if x[3]=='1' or x[3]=='5':
            hit_pos.append([int(x[2]),(int(int(x[0])*par.range+par.posx+k),int(int(x[1])*par.range+par.posy+k)),leb_n])
            cou+=1
        elif x[3]=='2' or x[3]=='6':
            hit_pos.append([int(x[2]),[int(int(x[0])*par.range+par.posx+k),int(int(x[1])*par.range+par.posy+k)],leb_n,int(eval(x[7])),0])
            x[5]=x[5].split('|')
            hit_pos[cou].append(x[5].pop(0))
            for i in x[5]:
                hit_pos[cou][1].append(int(int(i.split(':')[0])*par.range+par.posx+k))
                hit_pos[cou][1].append(int(int(i.split(':')[1])*par.range+par.posy+k))
            lenth=0
            if hit_pos[cou][5]=='B' or hit_pos[cou][5]=='L':
                for t in np.linspace(0,1,100):
                    if t>0:
                        dxyt=((B(t,hit_pos[cou][1][::2])-xt)**2+(B(t,hit_pos[cou][1][1::2])-yt)**2)**0.5
                        lenth+=dxyt
                    xt=B(t,hit_pos[cou][1][::2])
                    yt=B(t,hit_pos[cou][1][1::2])
            elif hit_pos[cou][5]=='P':
                P(0,hit_pos[cou][1][::2],hit_pos[cou][1][1::2])
                lenth=p_rad*abs(theta[0]-theta[2])
            hit_pos[cou][4]=lenth
            cou+=1
        last_pos=[int(x[0]),int(x[1])]
        last_time=int(x[2])
map_load()
######################initialize###########################
pg.init()
size={'x':(800,1200,3),'y':(1200,800)}
background_name='BG.jpg'
bg = pg.display.set_mode(size['y'])
pg.mouse.set_visible(False)
bg.fill((255,255,255))
wid=128
mouse_x, mouse_y = 0,0
start=0
label_number=1
end=20
clock=pg.time.Clock()
dfont = pg.font.SysFont("Arial", 30)


hit300=[]
hita=[]
body=[]
heart=[]


hitx=[]
hit100x=[]
hit300x=[]


score=0
score_run=0
combo=0
####################loading file#############################
if background_name  in os.listdir("sou\\"+song):
    k=0
    if np.shape(plt.imread("sou\\"+song+"\\"+background_name))[:2]!=size['x'][:2] and np.shape(plt.imread("sou\\"+song+"\\"+background_name))[0]>500:
        k=1
else:
    k=1
for name in os.listdir("sou\\"+song):
    if name.split('.')[-1] in ["jiff","jpeg","jpg","png","gif"] and k==1:
        img=plt.imread("sou\\"+song+"\\"+name,plt.IMREAD_UNCHANGED)
        cont=np.zeros(size['x'],dtype=np.uint8)
        if type(img[0][0][0])==np.uint8:
             k=0
        else:
            k=1
        lenx=len(img)
        leny=len(img[0])
        print(lenx,leny)
        print("strat loading")
        for i in range(size['x'][0]):
            for j in range(size['x'][1]):
                cont[i][j][:]=img[i*lenx//size['x'][0]][j*leny//size['x'][1]][:3]*(1+k*254)
        plt.imwrite("sou\\"+song+"\\"+background_name,cont)
        k=0
        reg=[]
    if name.split('.')[-1] =="mp3":
        pg.mixer.music.load("sou\\"+song+"\\"+name)
image = pg.image.load("sou\\"+song+"\\"+background_name).convert_alpha()
bg.blit(image,(0,0))
hit= pg.image.load_extended("sou\\skin\\object\\hitemp.png").convert_alpha()
#hit= pg.image.load_extended("sou\\skin\\object\\hitemp.png")
#hit=hit.convert_alpha()

hit100=pg.image.load_extended("sou\\skin\\object\\hit100.png").convert_alpha()

default=[]
for i in range(100):
    default.append(pg.image.load("sou\\skin\\number\\default-%d.png"%i).convert_alpha())
num=[]
for i in range(10):
    num.append(pg.image.load("sou//skin//number//score-%d@2x.png"%i).convert_alpha())
click= pg.mixer.Sound("sou\\skin\\soft-hitnormal.ogg")
combobreak=pg.mixer.Sound("sou\\skin//combobreak.ogg")
hit100sound=pg.mixer.Sound("sou\\skin//clap.wav")
mouse_cursor = pg.image.load("sou\\skin//cursor.png").convert_alpha()
hit0=pg.image.load("sou\\skin\\object\\hit0.png").convert_alpha()
for i in range(40):
    hit300.append(pg.image.load("sou\\skin\\object\\hit300@%dx.png"%i).convert_alpha())
    if i<30:
        hita.append(pg.image.load("sou\\skin\\object\\hit@%dx.png"%i).convert_alpha())
        body.append(pg.image.load("sou\\skin\\object\\body@%d.png"%i).convert_alpha())
        heart.append(pg.image.load("sou\\skin\\object\\heart@%d.png"%i).convert_alpha())

slider_body=body[29]
slider_heart= heart[29]
sliderfollowcircle=pg.image.load_extended("sou\\skin\\sliderfollowcircle.png").convert_alpha()
sliderb=pg.image.load_extended("sou\\skin\\sliderb.png").convert_alpha()

pg.display.update()
file=open("sou//data.txt",'w')
################################ start! ############################################
pg.mixer.music.play()
_time=int(time.time()*1000)
dat=0
dbt=0

while 1:
    __time=int(time.time()*1000)
    bg.blit(image,(0,0))
    #滑鼠的x,y座標
    mousepos= np.array(pg.mouse.get_pos())
    #隱藏滑鼠
    mousepos*=par.dpi
    mousepos-=64
    #用其他圖形代替滑鼠
    file.write("%f"%time.time()+'part0   \n')
    for i in range(end-1,start-1,-1):
        obj=hit_pos[i]
        dt=__time-_time-obj[0]
        if dt>-par.appear/par.apr-par.emerge:
            
            if len(obj[1])>2:
                slidetime=obj[3]-par.perfect
            else:
                slidetime=0
            if dt<=-par.appear/par.apr and dt>-par.appear/par.apr-par.emerge:
                place_hit_object(dt,0,obj)
            elif dt<par.perfect+slidetime:
                place_hit_object(dt,1,obj)
                if dt>0 and slidetime!=0:
                    if obj[5]=='P' and dt!=0:
                        slider_now_pos=P(dt/obj[3],[0,0,0],[0,0,0])
                    elif obj[5]=='B' or obj[5]=='L':
                        slider_now_pos=(B(dt/obj[3],obj[1][::2]),B(dt/obj[3],obj[1][1::2]))
                    bg.blit(sliderb,np.array(slider_now_pos)-wid/2)
                pg.draw.circle(bg,(100,150,40),(obj[1][0],obj[1][1]),64+(obj[0]>__time-_time)*(obj[0]-__time+_time)*par.apcsize*par.apr/par.appear,6)
                end=i+100
                if end>len(hit_pos):
                    end=len(hit_pos)
            else:
                start=i+1
            if __time-_time>obj[0]+par.perfect+slidetime:
                combobreak.play()
                combo=0
                hitx.append([i,obj[0]])
                hit_pos[i]=[obj[0]+0xFFFFFF,(obj[1][0],obj[1][1])]
    file.write("%f"%time.time()+'part1   \n')
    for i in hitx:
        if __time-_time<i[1]+par.label_fade and __time-_time>i[1]+par.perfect:
             bg.blit(hit0,hit_pos[i[0]][1])
        else:
            hitx.remove(i)
    for i in hit100x:
        if __time-_time<i[1]+par.label_fade:
             bg.blit(hit100,(hit_pos[i[0]][1]))
        else:
            hit100x.remove(i)
    for i in hit300x:
        if __time-i[1]<par.fade:
             bg.blit(hit300[39-int((__time-i[1])*40/par.fade)],(hit_pos[i[0]][1][0]-128,hit_pos[i[0]][1][1]-128))
        else:
            hit300x.remove(i)
            start=i[0]
    file.write("%f"%time.time()+'part2   \n')
    for n in range(7):
        bg.blit(num[((score_run)//(10**n))%10],(870+(7-n)*40,0))
    for i in range(len(str(combo))):
        bg.blit(num[(combo//(10**i))%10],((len(str(combo))-i)*30,720))
    if score>score_run:
        score_run+=(score-score_run)//10+1
    else:
        score_run=score
    clock.tick(900)
    text = dfont.render("%4d"%clock.get_fps(), 1, (255,255,255))
    bg.blit(text,(1100,750))
    bg.blit(mouse_cursor,mousepos)
    file.write("%f"%time.time()+'part0   \n')
    pg.display.update()
    file.write("%f"%time.time()+'part0   \n')
    for event in pg.event.get():
        if event.type == QUIT:
            file.close()
            pg.quit()
            sys.exit()
            file.close()
        elif event.type==KEYDOWN:
            key=event.key
            pos=np.array(pg.mouse.get_pos())*par.dpi
            for i in range(start,end):
                obj=hit_pos[i]
                if (key==K_a or key==K_s)and (pos[0]-obj[1][0])**2+(pos[1]-obj[1][1])**2<64**2:
                    dt=obj[0]-__time+_time
                    if dt>-par.perfect and dt<par.perfect:
                        hit300x.append([i,__time])
                        click.play()
                        score+=300*combo
                        hit_pos[i]=[obj[0]+0xFFFFFF,(obj[1][0],obj[1][1])]
                        combo+=1
                        break
                    elif dt>par.perfect and dt<par.good:
                        hit300x.append([i,__time])
                        hit100sound.play()
                        hit100x.append([i,obj[0]])
                        hit_pos[i]=[obj[0]+0xFFFFFF,(obj[1][0],obj[1][1])]
                        combo+=1
                        score+=100*combo
                        break
    file.write("%f"%time.time()+'part5   \n')
        

pygame 2.0.1 (SDL 2.0.14, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
1.  1057170 Tenjin Kotone - Chururira Chururira Daddadda!
2.  1083811 mafumafu - Rinne Tensei
3.  1090887 Sakuzyo - Imprinting
4.  1098181 umu - Dareka no Shinzou ni Nareta Nara
5.  1145949 LiSA - Shirushi (TV Size)
6.  1147361 Suzuki Masayuki - DADDY ! DADDY ! DO ! feat Suzuki Airi (TV Size)
7.  1157872 Feryquitous - I can avoid it#owo
8.  1182518 rerulili feat Dancing Dolls - Kami no Manima ni
9.  1206286 the peggies - Centimeter (TV Size)
10.  1211739 ReoNa - ANIMA (TV Size)
11.  1218852 YOASOBI - Yoru ni Kakeru
12.  1284534 Natsushiro Takaaki - Near feat Hatsune Miku
13.  1316672 40mP - The Flower of Raison D'etre (Cut Ver)
14.  1345462 Ueda Reina - Literature (TV Size)
15.  data.txt
16.  kuyamirai
17.  kuyamu tokai de mirai
18.  Teo
19.  test
chose song2
1. mafumafu - Rinne Tensei (AirinCat) [Hard].osu
2. mafumafu - Rinne Tensei (AirinCat) [Insane].osu
3. mafumafu - Rinn

SystemExit: 

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import matplotlib.pyplot as plt
img=np.

In [2]:
file=open("sou\\data.txt",'r')
x=file.read()
k=0
ar=[[],[],[],[],[]]
for j in x[:-9].split("part5   \n")[:-6]:
    j=j.split('   \n')
    for c in range(len(j)-1):
        j[c]=eval(j[c][:-5])
    j[len(j)-1]=eval(j[len(j)-1])
    ar[0].append(j[1]-j[0])
    ar[1].append(j[2]-j[1])
    ar[2].append(j[3]-j[2])
    ar[3].append(j[4]-j[3])
    ar[4].append(j[5]-j[4])
sums=0
for i in range(5):
    a=sum(ar[i])/len(ar[i])
    print("part[%d],"%i+"max",max(ar[i]),end=" av %f\n"%a)
    sums+=a
print(sums,1/sums)

part[0],max 0.027003049850463867 av 0.004850
part[1],max 0.004002094268798828 av 0.000655
part[2],max 0.002006053924560547 av 0.000778
part[3],max 0.0030291080474853516 av 0.000772
part[4],max 0.0030028820037841797 av 0.000250
0.007304407729960905 136.9036391408222


In [ ]:
part[0],max 0.030999183654785156 av 0.003644
part[1],max 0.005001068115234375 av 0.000807
part[2],max 0.0020139217376708984 av 0.000752
part[3],max 0.003998994827270508 av 0.000775
part[4],max 0.003999948501586914 av 0.000239
0.006217402798472362 160.83886349549422

In [ ]:
import pygame as pg, random, math, time

#建立球體
class Ball(pg.sprite.Sprite):
    dx = 0         #x位移量
    dy = 0         #y位移量
    x = 0          #球x坐標
    y = 0          #球y坐標
    direction = 0  #球移動方向
    speed = 0      #球移動速度
 
    def __init__(self, sp, srx, sry, radium, color):
        pg.sprite.Sprite.__init__(self)
        self.speed = sp
        self.x = srx
        self.y = sry
        #繪製球體
        self.image = pg.Surface([radium*2, radium*2])  
        self.image.fill((255,255,255))
        pg.draw.circle(self.image, color, (radium,radium), radium, 0)
        self.rect = self.image.get_rect()  #取得球體區域
        self.rect.center = (srx,sry)       #初始位置
        self.direction = random.randint(40,70)  #移動角度

 #球體移動 
    def update(self):         
        radian = math.radians(self.direction)    #角度轉為弳度
        self.dx = self.speed * math.cos(radian)  #球水平運動速度
        self.dy = -self.speed * math.sin(radian) #球垂直運動速度
        self.x += self.dx     #計算球新坐標
        self.y += self.dy
        self.rect.x = self.x  #移動球圖形
        self.rect.y = self.y
        #到達左右邊界
        if(self.rect.left <= 0 or self.rect.right >= screen.get_width()-10):  
            self.bouncelr()
        elif(self.rect.top <= 10):  #到達上邊界
            self.rect.top = 10
            self.bounceup()
        if(self.rect.bottom >= screen.get_height()-10):  #到達下邊界出界
            return True
        else:
            return False
 
    def bounceup(self):  #上邊界反彈
        self.direction = 360 - self.direction

    def bouncelr(self):  #左右邊界反彈
        self.direction = (180 - self.direction) % 360

#磚塊類別            
class Brick(pg.sprite.Sprite):
    def __init__(self, color, x, y):
        pg.sprite.Sprite.__init__(self)
        self.image = pg.Surface([38, 13])  #磚塊長寬38x13
        self.image.fill(color)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y

#板子類別
class Pad(pg.sprite.Sprite):
    def __init__(self):
        pg.sprite.Sprite.__init__(self)
        self.image = pg.image.load("sou//Teo\\hitcircleoverlay.png")  #滑板圖片
        self.image.convert()
        self.rect = self.image.get_rect()
        self.rect.x = int((screen.get_width() - self.rect.width)/2)  #滑板位置
        self.rect.y = screen.get_height() - self.rect.height - 30
        
#板子位置隨滑鼠移動 
    def update(self):  
        pos = pg.mouse.get_pos()  
        self.rect.x = pos[0]       #滑鼠x坐標
        #不要移出右邊界
        if self.rect.x > screen.get_width() - self.rect.width:
            self.rect.x = screen.get_width() - self.rect.width

#結束程式
def gameover(message): 
    global running
    #顯示訊息
    text = ffont.render(message, 1, (255,0,255))  
    screen.blit(text, (screen.get_width()/2-150,screen.get_height()/2-20))
    pg.display.update()  #更新畫面
    time.sleep(5)        #暫停5秒    
    running = False      #結束程式

pg.init()
score = 0  #得分
dfont = pg.font.SysFont("Arial", 20)    #下方訊息字體
ffont = pg.font.SysFont("SimHei", 32)   #結束程式訊息字體
soundhit = pg.mixer.Sound("sou//Teo\\clap.wav")  #碰到磚塊音效a
soundpad = pg.mixer.Sound("sou\\Teo//clap.wav")  #碰到滑板音效
#背景
screen = pg.display.set_mode((600, 400))
pg.display.set_caption("Sean's Brick Game")
background = pg.Surface(screen.get_size())
background = background.convert()
background.fill((255,255,255))
allsprite = pg.sprite.Group()  #建立全部角色群組
bricks = pg.sprite.Group()     #建立磚塊角色群組
ball = Ball(15, 300, 350, 10, (255,123,188)) #建立粉球
allsprite.add(ball)  #加入全部角色群組
pad = Pad()          #建立滑板球物件
allsprite.add(pad)   #加入全部角色群組

#建立磚塊
for row in range(0, 5):          #5列方塊
    for column in range(0, 15):  #每列15磚塊
        if row == 1 or row == 0: 
            brick = Brick((153,205,255), column * 40 + 1, row * 15 + 1)   #位置為40*15
        if row == 2: 
            brick = Brick((94,175,254), column * 40 + 1, row * 15 + 1)    
        if row == 3 or row == 4:  
            brick = Brick((52,153,207), column * 40 + 1, row * 15 + 1)  
        bricks.add(brick)     #加入磚塊角色群組
        allsprite.add(brick)  #加入全部角色群組
        
clock = pg.time.Clock()        
downmsg = "Press Left Click Button to start game!"  #起始訊息
playing = False  #開始時球不會移動
running = True

#運行的程式碼
while running:
    clock.tick(40)
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
    buttons = pg.mouse.get_pressed()  #檢查滑鼠按鈕
    if buttons[0]:            #按滑鼠左鍵後球可移動       
        playing = True
    textaa = dfont.render("%4d"%clock.get_fps(), 1, (0,0,0))
    screen.blit(textaa,(50,50))
    #遊戲進行中
    if playing == True:  
        screen.blit(background, (0,0))  #清除繪圖視窗
        fail = ball.update()  #移動球體
        if fail:              #球出界
            gameover("You failed!See you next time~")
        pad.update()          #更新滑板位置
        #檢查球和磚塊碰撞
        hitbrick = pg.sprite.spritecollide(ball, bricks, True)  
        if len(hitbrick) > 0:  #球和磚塊發生碰撞
            score += len(hitbrick)  #計算分數
            soundhit.play()    #球撞磚塊聲
            ball.rect.y += 20  #球向下移
            ball.bounceup()    #球反彈
            if len(bricks) == 0:  #所有磚塊消失
                gameover("Congratulations!!")
        #檢查球和滑板碰撞
        hitpad = pg.sprite.collide_rect(ball, pad)  
        if hitpad:  #球和滑板發生碰撞
            soundpad.play()  #球撞滑板聲
            ball.bounceup()  #球反彈
        allsprite.draw(screen)  #繪製所有角色
        downmsg = "Score: " + str(score)
    #繪製下方訊息    
    message = dfont.render(downmsg, 1, (255,0,255))
    screen.blit(message, (screen.get_width()/2-125,screen.get_height()-30))
    screen.blit(textaa,(50,50))
    pg.display.update()
pg.quit()